<table><tr align=left>
<td> <img src="swung_logo_vector.png" alt="Drawing" style="width: 450px;"/> </td>
<td> <img src="segysak_logo.png" alt="Drawing" style="width: 450px;"/> </td>
</tr></table>


# T21 Segysak Tutorial - Tony Hallam, April 2021



# Introduction

### What is `segysak`?

### Tour

### `segysak` vs `segyio` and `xarray`.

## What is `segysak`?

 - A library to make it easier to read and write segy data, as well as some sugar for common seismic related operations in Python.
 - 

## `segysak` Tour

 - **Repository:** https://github.com/trhallam/segysak
 - **ReadtheDocs:** https://segysak.readthedocs.io/en/latest/
   - Examples
   - API
 - **Slack:** https://swung.slack.com/messages/segysak/

## `segysak` vs `segyio` and `xarray`

# Installation

`pip install segysak`

Demo Data

Opening the Tutorial Notebook

In [6]:
from segysak.segy import segy_loader


# Basic Usage

 - Loading a 3D cube into your Notebook environment (How to examine the headers, how to load the data, how to filter headers for partial loading, explanation of bytes and determining these from the header DF, cover the segy_loader arguments).
 - Discuss the xarray format for seisnc. What does it offer. Xarray selection demonstration with plotting of iline/xline/slice.
 - Discussion of coordinates (local vs global), limitations and explanations.


# NetCDF File Format

 - Why use “another” format for seismic? Pro’s con’s when is this necessary?
 - Alternative approaches (e.g. save headers to hdf5 after scan).
 - Introduce segy_converter and cli `segysak convert`.


# Editing Data and Saving to SEG-Y

 - Apply a function to headers (adjust values as DF).
 - Apply a function to data (mask/filter).
 - Return to segy highlighting requirements of segysak.


# 10 Minute Break

In [1]:
import time
import tqdm

for i in tqdm.tqdm(range(60*10), desc="Seconds"):
    time.sleep(1)


Seconds: 100%|███████████████████████████████████████████████████████████████████████| 600/600 [10:03<00:00,  1.01s/it]


# Coordinate based selection

 - Example for an arbitrary line (explain limitations).
 - Example along a well path.


# Horizon extraction

 - Load a horizon and add it to a cube
 - Plotting on slices
 - Fill holes in a horizon
 - Masking and sampling around a horizon
 - Plotting maps


# Mapping functions over blocks

 - Learn how to use Xarray to map functions on blocks of data, such as trace maths.


# Vectorization of Seismic

 - I want to do machine learning and I need to tabularize my seismic and headers.
 - Now I need to send my results back to SEGY.


# Questions - Slack time because we will run over.

 - Fall backs to chat about memory management, dask, other file formats such as ZGY and Zarr
 - Demo of CLI for quick looks at headers or EBCIDC
 - Contribution Opportunities / Community led development
